In [1]:
# Pandas importeren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#import geopandas as gpd


# # Laad de gemeentegrenzen in
# gemeenten = gpd.read_file('bestuurlijkegrenzen.gpkg', layer='gemeenten')
# gemeenten = gemeenten.set_index('gemeentenaam')

def drop_columns(dataset, columns_to_remove):
    for column in columns_to_remove:
        if column in dataset.columns:
            dataset.drop(column, axis=1, inplace=True)

def clean_up_emtpy_datarows(dataset):
    dataset.drop(dataset[(dataset["Municipality_code"].isnull()) &
                         (dataset["Total_reported"] == 0) &
                         (dataset["Deceased"] == 0) &
                         (dataset["Hospital_admission"] == 0)]
                 .index, inplace=True)

def data_validity_check(dataset):
    print("Shape dataset = " + dataset.shape.__str__())
    dataset.to_csv('dataset.csv')
    print(dataset.isnull().sum())
    print("Shape alle data = " + dataset.shape.__str__())

    missing_values = dataset.isnull().sum()
    percent_missing = (missing_values / len(dataset)) * 100
    if any(percent_missing > 1):
        print("ERROR: Er zijn velden met meer dan 1% ontbrekende waarden.")
    else:
        print("Data validity check passed")

# Read datasets
datasets = {
    'aantallen_gemeente_per_dag': pd.concat([
        pd.read_csv('dataset/COVID-19_aantallen_gemeente_per_dag.csv', sep=';'),
        pd.read_csv('dataset/COVID-19_aantallen_gemeente_per_dag_tm_03102021.csv', sep=';')
    ]),
    'ziekenhuisopnames': pd.concat([
        pd.read_csv('dataset/COVID-19_ziekenhuisopnames.csv', sep=';'),
        pd.read_csv('dataset/COVID-19_ziekenhuisopnames_tm_03102021.csv', sep=';')
    ])
}

columns_to_remove = ["Date_of_report", "Version"]
date_columns = ['Date_of_publication', 'Date_of_statistics']
for name, dataset in datasets.items():
    drop_columns(dataset, columns_to_remove)
    for date_column in date_columns:
        if date_column in dataset:
            dataset.rename(columns={date_column : 'Date'}, inplace=True)
            dataset['Date'] = pd.to_datetime(dataset['Date']).dt.date
            dataset['Month'] = dataset['Date'].apply(lambda x: x.month)
            dataset['Year'] = dataset['Date'].apply(lambda x: x.year)


dataset = pd.merge(datasets['aantallen_gemeente_per_dag'], datasets['ziekenhuisopnames'],
                            on=['Date', 'Year', 'Month', 'Municipality_code', 'Municipality_name', 'Security_region_code', 'Security_region_name'])

clean_up_emtpy_datarows(dataset)

data_validity_check(dataset)

# #Laad shapefile in
# SHAPEFILE = 'cbsgebiedsindelingen2023.gpkg'

# cbs_gebiedsindeling = gpd.read_file(SHAPEFILE)
# cbs_gebiedsindeling.head()

Shape dataset = (384138, 14)
Date                                  0
Municipality_code                  1850
Municipality_name                  1850
Province                            899
Security_region_code               1850
Security_region_name               1850
Municipal_health_service            899
ROAZ_region                         899
Total_reported                        0
Deceased                              0
Month                                 0
Year                                  0
Hospital_admission_notification       0
Hospital_admission                    0
dtype: int64
Shape alle data = (384138, 14)
Data validity check passed


In [2]:
years = ['2020', '2021', '2022', '2023']
# creer een dictionary zodat de juiste dataset obv het ingevoerde jaar dynamisch kan worden geselecteerd
datasets = {}
for year in years:
    # Filter de data op basis van het jaar
    filtered_dataset = dataset[dataset['Year'] == int(year)]
    # Groepeer de data per provincie en gemeente en tel de aantallen op
    filtered_dataset = filtered_dataset[
        ['Hospital_admission', 'Deceased', 'Total_reported', 'Province', 'Municipality_name', 'Month', 'Year']].groupby(
        ['Province', 'Municipality_name', 'Month', 'Year'])[
        ['Hospital_admission', 'Deceased', 'Total_reported']].sum().reset_index().melt(
        id_vars=['Province', 'Municipality_name', 'Month', 'Year'], var_name='Type', value_name='Numbers')
    datasets[year] = filtered_dataset


# datasets['gemeenten'] = gemeenten

# corona_per_gemeente = datasets['2020'].groupby(['Municipality_name', 'Month', 'Year'])['Numbers'].sum().reset_index()
# corona_per_gemeente.head(20)

In [3]:
# Creëer een dictionary om de nummers naar maandnamen te mappen
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    
# de functie om de eerste screenshot van opdracht 2 te tonen
def show_national_graph(year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], show_months=False):
    
    if show_months == True: 
        national_data = datasets[year].groupby(['Month', 'Type'])['Numbers'].sum().reset_index()
        # Gebruik de map-methode van pandas om de maandnummers te vervangen door de corresponderende maandnamen
        national_data['Month'] = national_data['Month'].map(month_names)
        x_axisData = 'Month'
    else:
        national_data = datasets[year]
        x_axisData = 'Year'
    
    # toon de barplot conform de eerste screenshot van opdracht 2
    fig, ax = plt.subplots(figsize=(10, 5))
    g = sns.barplot(x = x_axisData,
            y = 'Numbers',
            hue = 'Type',
            hue_order=hue_order,
            estimator = np.sum,
            errorbar=None,
            data = national_data)
    
    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de eerste screenshot van opdracht 2 heeft een titel, daarom deze hier ook toegevoegd.
    g.set(title='The Netherlands in ' + year)

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=45)

    plt.show()
    
# de functie om de tweede screenshot van opdracht 2 te tonen
def show_provinces_graph(year='2020', hue_order=['Total_reported','Hospital_admission','Deceased']):
#     province_data = datasets[year].groupby(['Province', 'Type'], numeric_only=True).sum().reset_index()
    
    province_data = datasets[year].groupby(['Province', 'Type', 'Year'])['Numbers'].sum().reset_index()

    # zet de tijdelijke subset om naar province_2020 en past de pandas melt functie toe, ter voorbereiding van de visualisatie
    
    
    # toon de barplot conform de tweede screenshot van opdracht 2
    g = sns.barplot(x = 'Province',
                y = 'Numbers',
                hue = 'Type',
                hue_order=hue_order,
                data = province_data)

    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=45)

    plt.show()

def show_province_graph(location, year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], municipalities=False, show_months=False):
#     province_data = datasets[year].groupby(['Province', 'Type'], numeric_only=True).sum().reset_index()
    province_data = datasets[year]
    province_data = province_data[province_data['Province'] == location]
    if show_months == True:
        province_data = datasets[year].groupby(['Month', 'Type'])['Numbers'].sum().reset_index()
        
        # Gebruik de map-methode van pandas om de maandnummers te vervangen door de corresponderende maandnamen
        province_data['Month'] = province_data['Month'].map(month_names)
        x_axisData = 'Month'
    elif municipalities == False:
        province_data = province_data.groupby(['Province', 'Type', 'Year'])['Numbers'].sum().reset_index()
        x_axisData = 'Province'
    else:
        province_data = province_data.groupby(['Municipality_name', 'Type', 'Year'])['Numbers'].sum().reset_index()
        x_axisData = 'Municipality_name' 
    
   
    # toon de barplot conform de tweede screenshot van opdracht 2
    g = sns.barplot(x = x_axisData,
                    y = 'Numbers',
                    hue = 'Type',
                    hue_order=hue_order,
                    data = province_data)

    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=90)

    plt.show()

def show_heatmap(location, year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], show_months=False):
    heatmap_data = datasets[year]
    if location == 'Netherlands' or location == 'All provinces':
        heatmap_data = heatmap_data.pivot_table(values='Numbers', index=['Province'], columns='Month')
    else:
        heatmap_data = heatmap_data.pivot_table(values='Numbers', index=['Municipality_name'], columns='Month')
    # Create the heatmap
    sns.heatmap(heatmap_data, cmap="YlGnBu")

    # Set the title and labels for the plot
    plt.title("Total reported cases in 2020")
    plt.xlabel("Municipality")
    plt.ylabel("Province")

    # Show the plot
    plt.show()


In [4]:
# deze functie is de brug tussen de gebruikersinvoer en de visuatie die getoond moet worden
def event_handler(year='2020', total_reported=True, hospital_admission=True, deceased=True, grouping='Netherlands', municipalities=False, show_months=False):
    # Afhankelijk van de gebruikersinvoer, de juiste datasoort (Total_reported, Hospital_admission en Deceased) tonen
    hue_order=[]
    if total_reported:
        hue_order.append('Total_reported')
    if hospital_admission:
        hue_order.append('Hospital_admission')
    if deceased:
        hue_order.append('Deceased')

    # als geen opties zijn geselecteerd, dan alle opties tonen, om Python exception te voorkomen
    if len(hue_order) == 0:
        hue_order=['Total_reported','Hospital_admission','Deceased']
        
    # Afhankelijk van de gebruikersinvoer, de juiste groepering (landelijk of per province) tonen    
    if grouping =='Netherlands':
        show_national_graph(year,hue_order, show_months)
    elif grouping =='All provinces':
        show_provinces_graph(year,hue_order)
    else:
        show_province_graph(grouping, year, hue_order, municipalities, show_months)

        
# deze functie toont de GUI componenten en geeft de gebruikersinvoer door aan de event_handler

# dropdown opties om het te tonen jaar te selecteren
year_selector = widgets.Dropdown(
    options=datasets.keys(), # alleen de opties tonen waar datasets beschikbaar voor zijn
    value='2020',
    description='year',
    disabled=False
    )

# checkbox tonen voor Total_reported
total_reported_selector = widgets.Checkbox(
    value=True,
    description='Total_reported',
    disabled=False,
    indent=True
)

# checkbox tonen voor Hospital_admission
hospital_admission_selector = widgets.Checkbox(
    value=True,
    description='Hospital_admission',
    disabled=False,
    indent=True
)

# checkbox tonen voor Deceased
deceased_selector = widgets.Checkbox(
    value=True,
    description='Deceased',
    disabled=False,
    indent=True
)

# dropdown opties om het te tonen groepering te selecteren
provinces = list(datasets['2020']['Province'].unique())

grouping_selector = widgets.Dropdown(
    options=['Netherlands','All provinces'] + provinces,
    value='Netherlands',
    description='province',
    disabled=False
    )

municipalities_selector = widgets.Checkbox(
    value=False,
    description='Municipalities',
    disabled=False,
    indent=True,
    layout={'visibility': 'hidden'}
)

show_months_selector = widgets.Checkbox(
    value=False,
    description='Months',
    disabled=False,
    indent=True
)

# interactieve dataselectie en datavisualisatie
#widgets.interact(event_handler, year=year_selector,
#                 total_reported=total_reported_selector,
#                 hospital_admission=hospital_admission_selector,
#                 deceased=deceased_selector, grouping=grouping_selector,
#                 municipalities=municipalities_selector, show_months=show_months_selector)


# definieer de functie die wordt uitgevoerd wanneer de "show_months" checkbox wordt aangevinkt of uitgevinkt
# Als show_months op True is, set municipalities op False
def change_months_or_municipalites(change):
    if change['owner'].description == 'Municipalities' and change['owner'].value == True:
        show_months_selector.value = False
    elif change['owner'].description == 'Months' and change['owner'].value == True:
        municipalities_selector.value = False
show_months_selector.observe(change_months_or_municipalites, names='value')
municipalities_selector.observe(change_months_or_municipalites, names='value')

def on_grouping_selector_change(change):
    if grouping_selector.value in ['Netherlands']:
        municipalities_selector.layout.visibility = 'hidden'
        show_months_selector.layout.visibility = 'visible'
    elif grouping_selector.value in ['All provinces']:
        municipalities_selector.layout.visibility = 'hidden'
        show_months_selector.layout.visibility = 'hidden'
    else:
        municipalities_selector.layout.visibility = 'visible'
        show_months_selector.layout.visibility = 'visible'

grouping_selector.observe(on_grouping_selector_change, names='value')


def on_total_reported_selector_change(change):
    if change['owner'].value == False and hospital_admission_selector.value == False and deceased_selector.value == False:
        total_reported_selector.value = True
        
def on_hospital_admission_selector_change(change):
    if change['owner'].value == False and total_reported_selector.value == False and deceased_selector.value == False:
        hospital_admission_selector.value = True

def on_deceased_selector_change(change):
    if change['owner'].value == False and total_reported_selector.value == False and hospital_admission_selector.value == False: 
        deceased_selector.value = True


total_reported_selector.observe(on_total_reported_selector_change, names='value')
hospital_admission_selector.observe(on_hospital_admission_selector_change, names='value')
deceased_selector.observe(on_deceased_selector_change,names='value')


In [5]:
tab = widgets.Tab()

opdracht_tab = widgets.interactive(event_handler, year=year_selector, total_reported=total_reported_selector, hospital_admission=hospital_admission_selector, deceased=deceased_selector, grouping=grouping_selector, municipalities=municipalities_selector, show_months=show_months_selector)

tab.children = [widgets.VBox(children=opdracht_tab.children),widgets.VBox(children=[]),widgets.VBox(children=[])]
tab.set_title(0, 'Opdracht')
tab.set_title(1, 'Time series')
tab.set_title(2, 'Heatmap')

display(tab)
